In [10]:
import pickle
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from PIL import *
import tensorflow as tf
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
images = pickle.load( open("/content/drive/My Drive/Graduation Project/Graduation Project 1/Data Processing/data.pkl" ,"rb"))

In [9]:
labels = pickle.load( open("/content/drive/My Drive/Graduation Project/Graduation Project 1/Data Processing/labels.pkl" ,"rb"))

In [11]:
df = pd.read_csv("Data_Entry_2017_v2020.csv")
df.head(8)

FileNotFoundError: [Errno 2] No such file or directory: 'Data_Entry_2017_v2020.csv'

In [ ]:
df.drop(["OriginalImage[Width","Height]","OriginalImagePixelSpacing[x","y]"],axis = 1 ,inplace=True)

In [ ]:
# Cardiomegaly         2.48 %
# Emphysema            2.24 %
# Effusion             11.88 %
# No Finding           53.84 %
# Hernia               0.2 %      *****
# Infiltration         17.74 %
# Mass                 5.16 %
# Nodule               5.65 %
# Atelectasis          10.31 %
# Pneumothorax         4.73 %
# Pleural_Thickening   3.02 %
# Pneumonia            1.28 %     *****
# Fibrosis             1.5 %      *****
# Edema                2.05 %
# Consolidation        4.16 %

In [ ]:
df["labels_list"] = df["Finding Labels"].str.split("|")

In [ ]:
# get the row indices for the diseases
hernia_idx = df[df["labels_list"].apply(lambda x: "Hernia" in x and len(x) == 1)].index
Pneumonia_idx = df[df["labels_list"].apply(lambda x: "Pneumonia" in x and len(x) == 1)].index
Fibrosis_idx = df[df["labels_list"].apply(lambda x: "Fibrosis" in x and len(x) == 1)].index

In [ ]:
print(hernia_idx.shape[0])
print(Pneumonia_idx.shape[0])
print(Fibrosis_idx.shape[0])

In [ ]:
# get the images for the diseases
hernia_images = images[hernia_idx]
Pneumonia_images = images[Pneumonia_idx]
Fibrosis_images = images[Fibrosis_idx]

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs[0].imshow(hernia_images[0] ,cmap = "gray")
axs[0].set_title("Hernia")
axs[1].imshow(Pneumonia_images[0] ,cmap = "gray")
axs[1].set_title("Pneumonia")
axs[2].imshow(Fibrosis_images[0] ,cmap = "gray")
axs[2].set_title("Fibrosis")
plt.tight_layout()
plt.show()

In [ ]:
# Data Augmentation Layers
data_augmentation_net = Sequential([
    RandomZoom(height_factor=(-0.001, 0.001), width_factor=(-0.001, 0.001)),  # Random Zoom in or Zoom out with ± 0.001%
    RandomBrightness(factor=0.1),  # Random change the brightness (brighter or darker) by ± 1%
    RandomContrast(0.2) # Random change the contrast (the difference between the dark colors and bright colors) by ± 2%
])


In [ ]:
# Will convert the output from [data_augmentation_net] (which the output from it is tensor array) to numpy array
def convert_to_array(tensor):
    tensor = [np.array(tensor[i]) for i in range(len(tensor))]    # convert it to numpy array
    tensor = [cv.convertScaleAbs(tensor[i]) for i in range(len(tensor))] # convert the values in array to int and absolute
    return np.array(tensor)

In [ ]:
# Augmentation for the images
hernia_aug1 = data_augmentation_net(hernia_images, training=True)
hernia_aug1 = convert_to_array(hernia_aug1)

Pneumonia_aug1 = data_augmentation_net(Pneumonia_images, training=True)
Pneumonia_aug1 = convert_to_array(Pneumonia_aug1)

Fibrosis_aug1 = data_augmentation_net(Fibrosis_images, training=True)
Fibrosis_aug1 = convert_to_array(Fibrosis_aug1)

In [ ]:
print(hernia_aug1.shape)
print(Pneumonia_aug1.shape)
print(Fibrosis_aug1.shape)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
axs[0].imshow(hernia_images[5] ,cmap = "gray")
axs[0].set_title("Original Hernia Image")
axs[1].imshow(hernia_aug1[5] ,cmap = "gray")
axs[1].set_title("Augmented Hernia Image")
plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
axs[0].imshow(Pneumonia_images[2] ,cmap = "gray")
axs[0].set_title("Original Pneumonia Image")
axs[1].imshow(Pneumonia_aug1[2] ,cmap = "gray")
axs[1].set_title("Augmented Pneumonia Image")
plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
axs[0].imshow(Fibrosis_images[12] ,cmap = "gray")
axs[0].set_title("Original Fibrosis Image")
axs[1].imshow(Fibrosis_aug1[12] ,cmap = "gray")
axs[1].set_title("Augmented Fibrosis Image")
plt.tight_layout()
plt.show()

In [ ]:
# Second round of augmentation
hernia_aug2 = data_augmentation_net(hernia_aug1, training=True)
hernia_aug2 = convert_to_array(hernia_aug2)

Pneumonia_aug2 = data_augmentation_net(Pneumonia_aug1, training=True)
Pneumonia_aug2 = convert_to_array(Pneumonia_aug2)

Fibrosis_aug2 = data_augmentation_net(Fibrosis_aug1, training=True)
Fibrosis_aug2 = convert_to_array(Fibrosis_aug2)

In [ ]:
# Third round of augmentation
hernia_aug3 = data_augmentation_net(hernia_aug2, training=True)
hernia_aug3 = convert_to_array(hernia_aug3)

hernia_aug4 = data_augmentation_net(hernia_aug3, training=True)
hernia_aug4 = convert_to_array(hernia_aug4)

Pneumonia_aug3 = data_augmentation_net(Pneumonia_aug2, training=True)
Pneumonia_aug3 = convert_to_array(Pneumonia_aug3)

In [ ]:
# Create a dataframe for the augmented images
df_for_aug = pd.DataFrame(columns=['Finding Labels', 'Follow-up #', 'Patient ID','Patient Age', 'Patient Gender', 'View Position' ,"Image_array"])
interest_columns = ['Finding Labels', 'Follow-up #', 'Patient ID','Patient Age', 'Patient Gender', 'View Position']

Prepartion for the hernia dataframe

In [ ]:
df_for_hernia = pd.DataFrame(columns=['Finding Labels', 'Follow-up #', 'Patient ID','Patient Age', 'Patient Gender', 'View Position'])

In [ ]:
# Get the information (like the age,gender,etc) for the hernia images
df_for_hernia = pd.concat([df_for_hernia, df.loc[hernia_idx ,interest_columns]]).reset_index(drop=True)

In [ ]:
# Copy the information for all augmented images
df_for_hernia_copy = df_for_hernia.copy()
df_for_hernia = pd.concat([df_for_hernia, df_for_hernia_copy ,df_for_hernia_copy ,df_for_hernia_copy] ,axis = 0).reset_index(drop=True)

In [ ]:
# Store the images in a list
augmented_hernia_list = []

for i in range(len(hernia_aug1)):
    augmented_hernia_list.append({'Image_array': hernia_aug1[i]})

for i in range(len(hernia_aug2)):
    augmented_hernia_list.append({'Image_array': hernia_aug2[i]})

for i in range(len(hernia_aug3)):
    augmented_hernia_list.append({'Image_array': hernia_aug3[i]})

for i in range(len(hernia_aug4)):
    augmented_hernia_list.append({'Image_array': hernia_aug4[i]})

# Create a dataframe for hernia
hernia_df = pd.DataFrame(augmented_hernia_list)

In [ ]:
df_for_hernia = pd.concat([df_for_hernia ,hernia_df] ,axis = 1)

In [ ]:
df_for_hernia.head()

Prepartion for the Pneumonia dataframe

In [ ]:
df_for_Pneumonia = pd.DataFrame(columns=['Finding Labels', 'Follow-up #', 'Patient ID','Patient Age', 'Patient Gender', 'View Position'])

In [ ]:
df_for_Pneumonia = pd.concat([df_for_Pneumonia, df.loc[Pneumonia_idx ,interest_columns]]).reset_index(drop=True)

In [ ]:
df_for_Pneumonia_copy = df_for_Pneumonia.copy()
df_for_Pneumonia = pd.concat([df_for_Pneumonia, df_for_Pneumonia_copy ,df_for_Pneumonia_copy] ,axis = 0).reset_index(drop=True)

In [ ]:
augmented_Pneumonia_list = []

for i in range(len(Pneumonia_aug1)):
    augmented_Pneumonia_list.append({'Image_array': Pneumonia_aug1[i]})

for i in range(len(Pneumonia_aug2)):
    augmented_Pneumonia_list.append({'Image_array': Pneumonia_aug2[i]})

for i in range(len(Pneumonia_aug3)):
    augmented_Pneumonia_list.append({'Image_array': Pneumonia_aug3[i]})

Pneumonia_df = pd.DataFrame(augmented_Pneumonia_list)

In [ ]:
df_for_Pneumonia = pd.concat([df_for_Pneumonia ,Pneumonia_df] ,axis = 1)

Prepartion for the Fibrosis  dataframe

In [ ]:
df_for_Fibrosis = pd.DataFrame(columns=['Finding Labels', 'Follow-up #', 'Patient ID','Patient Age', 'Patient Gender', 'View Position'])

In [ ]:
df_for_Fibrosis = pd.concat([df_for_Fibrosis, df.loc[Fibrosis_idx ,interest_columns]]).reset_index(drop=True)

In [ ]:
df_for_Fibrosis_copy = df_for_Fibrosis.copy()
df_for_Fibrosis = pd.concat([df_for_Fibrosis, df_for_Fibrosis_copy] ,axis = 0).reset_index(drop=True)

In [ ]:
augmented_Fibrosis_list = []

for i in range(len(Fibrosis_aug1)):
    augmented_Fibrosis_list.append({'Image_array': Fibrosis_aug1[i]})

for i in range(len(Fibrosis_aug2)):
    augmented_Fibrosis_list.append({'Image_array': Fibrosis_aug2[i]})

Fibrosis_df = pd.DataFrame(augmented_Fibrosis_list)

In [ ]:
df_for_Fibrosis = pd.concat([df_for_Fibrosis ,Fibrosis_df] ,axis = 1)

In [ ]:
df_for_Fibrosis.shape,df_for_hernia.shape,df_for_Pneumonia.shape

In [ ]:
# Concat all the dataframes in one dataframe
df_for_aug = pd.concat([df_for_Fibrosis,df_for_hernia,df_for_Pneumonia] ,axis = 0).reset_index(drop=True)

In [ ]:
df_for_aug